# ETL Pipeline - Freelance Job Earnings Data Warehouse

This notebook extracts data from raw sources, transforms it into a star schema, and loads it into dimension and fact tables.

## Output Tables:
- **fact_job_earnings.csv** - Fact table with job earnings metrics
- **dim_worker.csv** - Worker dimension
- **dim_platform.csv** - Platform dimension
- **dim_region.csv** - Region dimension
- **dim_project.csv** - Project dimension
- **dim_date.csv** - Date dimension

In [1]:
import pandas as pd
import json
import os
from pathlib import Path

## 1. Configuration

In [2]:
# Define paths
BASE_DIR = Path.cwd().parent
RAW_DATA_DIR = BASE_DIR / 'data_raw'
CLEANED_DATA_DIR = BASE_DIR / 'data_cleaned'

# Create output directory if it doesn't exist
CLEANED_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Raw data directory: {RAW_DATA_DIR}")
print(f"Cleaned data directory: {CLEANED_DATA_DIR}")

Raw data directory: C:\Users\KilZonR\Desktop\frelance Final\data_raw
Cleaned data directory: C:\Users\KilZonR\Desktop\frelance Final\data_cleaned


---
## 2. EXTRACT - Load Raw Data

In [3]:
# Extract jobs transactions (main facts data)
jobs_df = pd.read_csv(RAW_DATA_DIR / 'jobs_transactions.csv')
print(f"Jobs transactions: {len(jobs_df)} records")
jobs_df.head()

Jobs transactions: 1950 records


,job_id,worker_id,platform,client_region,project_type,job_category,payment_method,work_date,earnings_usd,job_completed,job_duration_days,hourly_rate,job_success_rate,client_rating,rehire_rate,marketing_spend
0,1,1,Fiverr,Asia,Fixed,Web Development,Mobile Banking,2025-11-16,1620,180,1,95.79,68.73,3.18,40.19,53
1,2,2,Fiverr,Australia,Fixed,App Development,Mobile Banking,2025-12-15,9078,218,54,86.38,97.54,3.44,36.53,486
2,3,3,Fiverr,UK,Hourly,Web Development,Crypto,2025-08-19,3455,27,46,85.17,86.60,4.20,74.05,489
3,4,4,PeoplePerHour,Asia,Hourly,Data Entry,Bank Transfer,2025-09-18,5577,17,41,14.37,79.93,4.47,27.58,67
4,5,5,Upwork,Asia,Hourly,Digital Marketing,Crypto,2025-03-11,5898,245,41,99.37,57.80,5.00,69.09,489


In [4]:
# Extract workers dimension (JSON)
with open(RAW_DATA_DIR / 'workers.json', 'r') as f:
    workers_raw = json.load(f)
workers_df = pd.DataFrame(workers_raw)
print(f"Workers: {len(workers_df)} records")
workers_df.head()

Workers: 1950 records


,worker_id,experience_level,primary_skill
0,1,Beginner,Web Development
1,2,Beginner,App Development
2,3,Beginner,Web Development
3,4,Intermediate,Data Entry
4,5,Expert,Digital Marketing


In [5]:
# Extract platforms dimension (CSV)
platforms_df = pd.read_csv(RAW_DATA_DIR / 'platforms.csv')
print(f"Platforms: {len(platforms_df)} records")
platforms_df

Platforms: 5 records


,platform_id,platform_name,category,payment_cycle
0,1,Fiverr,Freelancing,Crypto
1,2,Peopleperhour,Freelancing,Bank Transfer
2,3,Upwork,Freelancing,PayPal
3,4,Toptal,Freelancing,Crypto
4,5,Freelancer,Freelancing,Crypto


In [6]:
# Extract regions dimension (JSON)
with open(RAW_DATA_DIR / 'regions.json', 'r') as f:
    regions_raw = json.load(f)
regions_df = pd.DataFrame(regions_raw)
print(f"Regions: {len(regions_df)} records")
regions_df

Regions: 7 records


,region_id,region,cost_of_living_index
0,1,Asia,112.0
1,2,Australia,120.0
2,3,Uk,102.0
3,4,Europe,109.0
4,5,Usa,97.0
5,6,Middle East,128.0
6,7,Canada,124.0


In [7]:
# Extract projects dimension (CSV)
projects_df = pd.read_csv(RAW_DATA_DIR / 'projects.csv')
print(f"Projects: {len(projects_df)} records")
projects_df.head(10)

Projects: 16 records


,project_id,project_type,job_category
0,1,Fixed,Web Development
1,2,Fixed,App Development
2,3,Hourly,Web Development
3,4,Hourly,Data Entry
4,5,Hourly,Digital Marketing
5,6,Fixed,Customer Support
6,7,Fixed,Data Entry
7,8,Fixed,Content Writing
8,9,Hourly,App Development
9,10,Hourly,Customer Support


In [9]:
# Extract date dimension (CSV)
dim_date_df = pd.read_csv(RAW_DATA_DIR / 'dates.csv')
print(f"Dates: {len(dim_date_df)} records")
dim_date_df.head()

Dates: 365 records


,date_id,full_date,year,quarter,month_number,month_name,day_of_week,is_weekend,is_holiday
0,20250101,2025-01-01,2025,1,1,January,Wednesday,0,1
1,20250102,2025-01-02,2025,1,1,January,Thursday,0,0
2,20250103,2025-01-03,2025,1,1,January,Friday,0,0
3,20250104,2025-01-04,2025,1,1,January,Saturday,1,0
4,20250105,2025-01-05,2025,1,1,January,Sunday,1,0


---
## 3. TRANSFORM - Create Dimension Tables

### 3.1 dim_worker

In [10]:
# Transform worker dimension
dim_worker = workers_df.copy()

# Ensure proper column order as per data dictionary
dim_worker = dim_worker[['worker_id', 'experience_level', 'primary_skill']]

# Convert worker_id to string (as per data dictionary)
dim_worker['worker_id'] = dim_worker['worker_id'].astype(str)

print(f"dim_worker shape: {dim_worker.shape}")
dim_worker.head()

dim_worker shape: (1950, 3)


,worker_id,experience_level,primary_skill
0,1,Beginner,Web Development
1,2,Beginner,App Development
2,3,Beginner,Web Development
3,4,Intermediate,Data Entry
4,5,Expert,Digital Marketing


### 3.2 dim_platform

In [11]:
# Transform platform dimension
dim_platform = platforms_df.copy()

# Standardize platform names for matching with jobs data
# Create a mapping for case-insensitive matching
dim_platform['platform_name_standardized'] = dim_platform['platform_name'].str.title()

# Rename to match data dictionary
dim_platform = dim_platform.rename(columns={'payment_cycle': 'payment_cycle'})

# Ensure proper column order
dim_platform = dim_platform[['platform_id', 'platform_name', 'category', 'payment_cycle']]

print(f"dim_platform shape: {dim_platform.shape}")
dim_platform

dim_platform shape: (5, 4)


,platform_id,platform_name,category,payment_cycle
0,1,Fiverr,Freelancing,Crypto
1,2,Peopleperhour,Freelancing,Bank Transfer
2,3,Upwork,Freelancing,PayPal
3,4,Toptal,Freelancing,Crypto
4,5,Freelancer,Freelancing,Crypto


### 3.3 dim_region

In [12]:
# Transform region dimension
dim_region = regions_df.copy()

# Ensure proper column order as per data dictionary
dim_region = dim_region[['region_id', 'region', 'cost_of_living_index']]

print(f"dim_region shape: {dim_region.shape}")
dim_region

dim_region shape: (7, 3)


,region_id,region,cost_of_living_index
0,1,Asia,112.0
1,2,Australia,120.0
2,3,Uk,102.0
3,4,Europe,109.0
4,5,Usa,97.0
5,6,Middle East,128.0
6,7,Canada,124.0


### 3.4 dim_project

In [13]:
# Transform project dimension
dim_project = projects_df.copy()

# Ensure proper column order as per data dictionary
dim_project = dim_project[['project_id', 'project_type', 'job_category']]

print(f"dim_project shape: {dim_project.shape}")
dim_project

dim_project shape: (16, 3)


,project_id,project_type,job_category
0,1,Fixed,Web Development
1,2,Fixed,App Development
2,3,Hourly,Web Development
3,4,Hourly,Data Entry
4,5,Hourly,Digital Marketing
5,6,Fixed,Customer Support
6,7,Fixed,Data Entry
7,8,Fixed,Content Writing
8,9,Hourly,App Development
9,10,Hourly,Customer Support


### 3.5 dim_date

In [14]:
# Transform date dimension
dim_date = dim_date_df.copy()

# Ensure proper column order as per data dictionary
dim_date = dim_date[['date_id', 'full_date', 'day_of_week', 'is_weekend', 'is_holiday', 
                      'month_name', 'month_number', 'quarter', 'year']]

print(f"dim_date shape: {dim_date.shape}")
dim_date.head()

dim_date shape: (365, 9)


,date_id,full_date,day_of_week,is_weekend,is_holiday,month_name,month_number,quarter,year
0,20250101,2025-01-01,Wednesday,0,1,January,1,1,2025
1,20250102,2025-01-02,Thursday,0,0,January,1,1,2025
2,20250103,2025-01-03,Friday,0,0,January,1,1,2025
3,20250104,2025-01-04,Saturday,1,0,January,1,1,2025
4,20250105,2025-01-05,Sunday,1,0,January,1,1,2025


---
## 4. TRANSFORM - Create Fact Table

In [15]:
# Start with jobs transactions
fact_job_earnings = jobs_df.copy()

# Convert work_date to datetime for joining
fact_job_earnings['work_date'] = pd.to_datetime(fact_job_earnings['work_date'])

print(f"Initial fact table: {len(fact_job_earnings)} records")
fact_job_earnings.head()

Initial fact table: 1950 records


,job_id,worker_id,platform,client_region,project_type,job_category,payment_method,work_date,earnings_usd,job_completed,job_duration_days,hourly_rate,job_success_rate,client_rating,rehire_rate,marketing_spend
0,1,1,Fiverr,Asia,Fixed,Web Development,Mobile Banking,2025-11-16,1620,180,1,95.79,68.73,3.18,40.19,53
1,2,2,Fiverr,Australia,Fixed,App Development,Mobile Banking,2025-12-15,9078,218,54,86.38,97.54,3.44,36.53,486
2,3,3,Fiverr,UK,Hourly,Web Development,Crypto,2025-08-19,3455,27,46,85.17,86.60,4.20,74.05,489
3,4,4,PeoplePerHour,Asia,Hourly,Data Entry,Bank Transfer,2025-09-18,5577,17,41,14.37,79.93,4.47,27.58,67
4,5,5,Upwork,Asia,Hourly,Digital Marketing,Crypto,2025-03-11,5898,245,41,99.37,57.80,5.00,69.09,489


In [16]:
# Create date_id from work_date (YYYYMMDD format)
fact_job_earnings['date_id'] = fact_job_earnings['work_date'].dt.strftime('%Y%m%d').astype(int)

fact_job_earnings[['work_date', 'date_id']].head()

,work_date,date_id
0,2025-11-16,20251116
1,2025-12-15,20251215
2,2025-08-19,20250819
3,2025-09-18,20250918
4,2025-03-11,20250311


In [17]:
# Create platform lookup for getting platform_id
# Handle case differences (e.g., 'PeoplePerHour' vs 'Peopleperhour')
platform_lookup = dim_platform[['platform_id', 'platform_name']].copy()
platform_lookup['platform_name_lower'] = platform_lookup['platform_name'].str.lower()

fact_job_earnings['platform_lower'] = fact_job_earnings['platform'].str.lower()

# Merge to get platform_id
fact_job_earnings = fact_job_earnings.merge(
    platform_lookup[['platform_id', 'platform_name_lower']],
    left_on='platform_lower',
    right_on='platform_name_lower',
    how='left'
)

# Drop temporary columns
fact_job_earnings = fact_job_earnings.drop(columns=['platform_lower', 'platform_name_lower'])

print(f"Platform ID mapping check (null count): {fact_job_earnings['platform_id'].isna().sum()}")

Platform ID mapping check (null count): 0


In [18]:
# Create region lookup for getting region_id
# Handle case differences (e.g., 'UK' vs 'Uk', 'USA' vs 'Usa')
region_lookup = dim_region[['region_id', 'region']].copy()
region_lookup['region_lower'] = region_lookup['region'].str.lower()

fact_job_earnings['client_region_lower'] = fact_job_earnings['client_region'].str.lower()

# Merge to get region_id
fact_job_earnings = fact_job_earnings.merge(
    region_lookup[['region_id', 'region_lower']],
    left_on='client_region_lower',
    right_on='region_lower',
    how='left'
)

# Drop temporary columns
fact_job_earnings = fact_job_earnings.drop(columns=['client_region_lower', 'region_lower'])

print(f"Region ID mapping check (null count): {fact_job_earnings['region_id'].isna().sum()}")

Region ID mapping check (null count): 0


In [19]:
# Create project lookup for getting project_id
# Need to match on both project_type and job_category
project_lookup = dim_project[['project_id', 'project_type', 'job_category']].copy()
project_lookup['project_type_lower'] = project_lookup['project_type'].str.lower()
project_lookup['job_category_lower'] = project_lookup['job_category'].str.lower()

fact_job_earnings['project_type_lower'] = fact_job_earnings['project_type'].str.lower()
fact_job_earnings['job_category_lower'] = fact_job_earnings['job_category'].str.lower()

# Merge to get project_id
fact_job_earnings = fact_job_earnings.merge(
    project_lookup[['project_id', 'project_type_lower', 'job_category_lower']],
    on=['project_type_lower', 'job_category_lower'],
    how='left'
)

# Drop temporary columns
fact_job_earnings = fact_job_earnings.drop(columns=['project_type_lower', 'job_category_lower'])

print(f"Project ID mapping check (null count): {fact_job_earnings['project_id'].isna().sum()}")

Project ID mapping check (null count): 0


In [20]:
# Convert worker_id to string for consistency with dim_worker
fact_job_earnings['worker_id'] = fact_job_earnings['worker_id'].astype(str)

# Create is_gap_day flag: 1 if earnings are zero/missing or job_completed is 0, else 0
fact_job_earnings['is_gap_day'] = (
    (fact_job_earnings['earnings_usd'].isna()) | 
    (fact_job_earnings['earnings_usd'] == 0) | 
    (fact_job_earnings['job_completed'] == 0)
).astype(int)

print(f"is_gap_day distribution:\n{fact_job_earnings['is_gap_day'].value_counts()}")

is_gap_day distribution:
is_gap_day
0    1950
Name: count, dtype: int64


In [21]:
# Select and order columns for final fact table as per data dictionary
fact_columns = [
    'job_id',           # DK - Degenerate key
    'worker_id',        # FK -> dim_worker
    'platform_id',      # FK -> dim_platform
    'region_id',        # FK -> dim_region
    'project_id',       # FK -> dim_project
    'date_id',          # FK -> dim_date
    'earnings_usd',     # Metric
    'job_completed',    # Metric
    'job_duration_days',# Metric
    'hourly_rate',      # Metric
    'job_success_rate', # Metric
    'client_rating',    # Metric
    'rehire_rate',      # Metric
    'marketing_spend',  # Metric
    'is_gap_day'        # Calculated flag
]

fact_job_earnings_final = fact_job_earnings[fact_columns].copy()

# Convert FK columns to int (handling NaN)
for col in ['platform_id', 'region_id', 'project_id']:
    fact_job_earnings_final[col] = fact_job_earnings_final[col].fillna(-1).astype(int)
    # Replace -1 back to NaN for cleaner output (optional)
    fact_job_earnings_final[col] = fact_job_earnings_final[col].replace(-1, pd.NA)

print(f"Final fact_job_earnings shape: {fact_job_earnings_final.shape}")
fact_job_earnings_final.head(10)

Final fact_job_earnings shape: (1950, 15)


,job_id,worker_id,platform_id,region_id,project_id,date_id,earnings_usd,job_completed,job_duration_days,hourly_rate,job_success_rate,client_rating,rehire_rate,marketing_spend,is_gap_day
0,1,1,1,1,1,20251116,1620,180,1,95.79,68.73,3.18,40.19,53,0
1,2,2,1,2,2,20251215,9078,218,54,86.38,97.54,3.44,36.53,486,0
2,3,3,1,3,3,20250819,3455,27,46,85.17,86.60,4.20,74.05,489,0
3,4,4,2,1,4,20250918,5577,17,41,14.37,79.93,4.47,27.58,67,0
4,5,5,3,1,5,20250311,5898,245,41,99.37,57.80,5.00,69.09,489,0
5,6,6,4,4,6,20250711,6867,280,8,43.04,57.80,4.87,43.88,290,0
6,7,7,1,5,3,20250920,1677,96,32,20.50,52.90,4.29,45.99,343,0
7,8,8,4,2,7,20250815,6193,112,30,82.15,93.31,3.84,31.59,168,0
8,9,9,4,5,8,20250126,8446,233,46,26.44,80.06,4.27,50.11,396,0
9,10,10,2,6,4,20250519,6608,156,52,54.99,85.40,4.57,32.76,160,0


---
## 5. Data Quality Checks

In [22]:
# Check for any unmapped foreign keys
print("=== Data Quality Report ===")
print(f"\nFact Table Records: {len(fact_job_earnings_final)}")
print(f"\nNull/Unmapped Foreign Keys:")
print(f"  - platform_id: {fact_job_earnings_final['platform_id'].isna().sum()}")
print(f"  - region_id: {fact_job_earnings_final['region_id'].isna().sum()}")
print(f"  - project_id: {fact_job_earnings_final['project_id'].isna().sum()}")
print(f"\nDimension Tables:")
print(f"  - dim_worker: {len(dim_worker)} records")
print(f"  - dim_platform: {len(dim_platform)} records")
print(f"  - dim_region: {len(dim_region)} records")
print(f"  - dim_project: {len(dim_project)} records")
print(f"  - dim_date: {len(dim_date)} records")

=== Data Quality Report ===

Fact Table Records: 1950

Null/Unmapped Foreign Keys:
  - platform_id: 0
  - region_id: 0
  - project_id: 0

Dimension Tables:
  - dim_worker: 1950 records
  - dim_platform: 5 records
  - dim_region: 7 records
  - dim_project: 16 records
  - dim_date: 365 records


In [23]:
# Verify referential integrity for worker_id
workers_in_fact = set(fact_job_earnings_final['worker_id'].unique())
workers_in_dim = set(dim_worker['worker_id'].unique())

orphan_workers = workers_in_fact - workers_in_dim
print(f"Workers in fact but not in dimension: {len(orphan_workers)}")
if orphan_workers:
    print(f"  Sample orphan worker IDs: {list(orphan_workers)[:5]}")

Workers in fact but not in dimension: 0


---
## 6. LOAD - Save to data_cleaned folder

In [24]:
# Save dimension tables
dim_worker.to_csv(CLEANED_DATA_DIR / 'dim_worker.csv', index=False)
print(f"✓ Saved dim_worker.csv ({len(dim_worker)} records)")

dim_platform.to_csv(CLEANED_DATA_DIR / 'dim_platform.csv', index=False)
print(f"✓ Saved dim_platform.csv ({len(dim_platform)} records)")

dim_region.to_csv(CLEANED_DATA_DIR / 'dim_region.csv', index=False)
print(f"✓ Saved dim_region.csv ({len(dim_region)} records)")

dim_project.to_csv(CLEANED_DATA_DIR / 'dim_project.csv', index=False)
print(f"✓ Saved dim_project.csv ({len(dim_project)} records)")

dim_date.to_csv(CLEANED_DATA_DIR / 'dim_date.csv', index=False)
print(f"✓ Saved dim_date.csv ({len(dim_date)} records)")

✓ Saved dim_worker.csv (1950 records)
✓ Saved dim_platform.csv (5 records)
✓ Saved dim_region.csv (7 records)
✓ Saved dim_project.csv (16 records)
✓ Saved dim_date.csv (365 records)


In [25]:
# Save fact table
fact_job_earnings_final.to_csv(CLEANED_DATA_DIR / 'fact_job_earnings.csv', index=False)
print(f"✓ Saved fact_job_earnings.csv ({len(fact_job_earnings_final)} records)")

✓ Saved fact_job_earnings.csv (1950 records)


In [26]:
# List all output files
print("\n=== ETL Pipeline Complete ===")
print(f"\nOutput files in {CLEANED_DATA_DIR}:")
for file in sorted(CLEANED_DATA_DIR.glob('*.csv')):
    size = file.stat().st_size / 1024  # KB
    print(f"  - {file.name} ({size:.1f} KB)")


=== ETL Pipeline Complete ===

Output files in C:\Users\KilZonR\Desktop\frelance Final\data_cleaned:
  - dim_date.csv (17.7 KB)
  - dim_platform.csv (0.2 KB)
  - dim_project.csv (0.4 KB)
  - dim_region.csv (0.1 KB)
  - dim_worker.csv (55.7 KB)
  - fact_job_earnings.csv (124.1 KB)


---
## 7. Summary Statistics

In [27]:
# Display summary statistics for the fact table
print("=== Fact Table Summary Statistics ===")
fact_job_earnings_final[['earnings_usd', 'job_completed', 'job_duration_days', 
                          'hourly_rate', 'job_success_rate', 'client_rating', 
                          'rehire_rate', 'marketing_spend']].describe()

=== Fact Table Summary Statistics ===


,earnings_usd,job_completed,job_duration_days,hourly_rate,job_success_rate,client_rating,rehire_rate,marketing_spend
count,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000
mean,5017.566667,150.864103,44.704615,52.579774,74.951846,3.995892,44.556913,248.517436
std,2926.279918,85.480770,26.022998,26.927584,14.615735,0.575976,20.193842,148.084581
min,51.000000,5.000000,1.000000,5.020000,50.160000,3.000000,10.000000,0.000000
25%,2419.000000,76.000000,22.000000,30.047500,61.917500,3.510000,27.150000,119.000000
50%,5048.000000,149.000000,45.000000,52.285000,75.400000,3.990000,43.920000,252.500000
75%,7608.250000,225.000000,67.000000,75.125000,87.537500,4.500000,61.690000,379.000000
max,9991.000000,299.000000,89.000000,99.830000,99.990000,5.000000,79.950000,499.000000
